In [1]:
import wget
wget.download("https://plus.unsplash.com/premium_photo-1677541205130-51e60e937318?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1000&q=80", "./temp_data/img_3.jpg")

In [44]:
import cv2
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

In [45]:
def resize_watermark(img, watermark):
    w_height, w_width = watermark.shape[:2]
    ratio = w_width / w_height

    new_height = img.shape[0] // 7
    new_width = int(new_height * ratio)

    resized_watermark = cv2.resize(watermark, (new_width, new_height))

    return resized_watermark

In [46]:
def change_gray(watermark):
    gray_watermark = np.where(0<watermark, 127, watermark)
    return gray_watermark

In [47]:
def random_alpha_modify(tiled_pattern):
    height, width, _ = tiled_pattern.shape
    x = np.random.randint(0, int(width*0.7))
    y = np.random.randint(0, int(height*0.7))
    h = height//2
    w = width//2

    tiled_pattern[y:y+h, x:x+w, 3] *= 0.7

    return tiled_pattern

In [48]:
def visualize_numpy(np_array):
    img = Image.fromarray(np_array.astype(np.uint8))
    return img

In [49]:
def overlay(image, watermark, alpha_range=(0.08, 0.2), alpha_crop=True, gray=True):
    if gray==True:
        watermark = change_gray(watermark)
    
    watermark = resize_watermark(image, watermark)

    tile_x = image.shape[1] // watermark.shape[1] + 1
    tile_y = image.shape[0] // watermark.shape[0] + 1

    tiled_pattern = np.tile(watermark, (tile_y, tile_x, 1))
    tiled_pattern = tiled_pattern[:image.shape[0], :image.shape[1]]

    if alpha_crop==True:
        tiled_pattern = random_alpha_modify(tiled_pattern)

    pattern_bgr = tiled_pattern[..., :3]
    pattern_alpha = tiled_pattern[..., 3]

    alpha_mask = cv2.merge([pattern_alpha, pattern_alpha, pattern_alpha]) / 255.0 * np.random.uniform(*alpha_range)

    overlay = pattern_bgr * alpha_mask + image * (1 - alpha_mask)
    overlay = overlay.astype(np.uint8)

    return overlay

In [55]:
def main(watermark_dir, image_dir, save_dir, overlay_iter=3):
    
    watermark_paths = [(os.path.join(watermark_dir, cur), os.path.splitext(cur)[0]) for cur in os.listdir(watermark_dir)]
    image_paths = [(os.path.join(image_dir, cur), os.path.splitext(cur)[0]) for cur in os.listdir(image_dir)]

    for image_path, img_fn in tqdm(image_paths, total=len(image_paths), leave=True):
        image = cv2.imread(image_path).astype(np.float64)

        for watermark_path, watermark_fn in watermark_paths:
            watermark = cv2.imread(watermark_path, -1).astype(np.float64)

            for idx in range(overlay_iter):
                # if np.random.randint(0,10) % 2 == 0:
                #     output = overlay(image, watermark, (0.05, 0.18), gray=True)
                # else:
                output = overlay(image, watermark, (0.08, 0.2), gray=False)
                
                output = output.astype(np.uint8)
                cv2.imwrite(os.path.join(save_dir, f"{img_fn}_{watermark_fn}_{idx}.png"), output)

In [56]:
main("./watermark", "./temp_data", "./result", overlay_iter=1)

100%|██████████| 2/2 [01:51<00:00, 55.93s/it]


In [57]:
a = [1,2,3]
b = [3,2,1]

In [61]:
list(zip(a,b,))

[(1, 3), (2, 2), (3, 1)]